<a href="https://colab.research.google.com/github/dxda6216/SPARK_fluorescence_anisotropy/blob/main/SPARK_fluorescence_anisotropy_data_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###
### This script is to pre-process a SPARK fluorescence anisotropy data file (Excel file).
###
import pandas as pd
from datetime import datetime, timedelta, timezone
import io
import os
from google.colab import files
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

#@markdown **This script is to pre-process a SPARK fluorescence anisotropy data file (Excel file).**

#@markdown This script only works with a specific format of Excel files generated by Tecan SPARK multimode microplate reader. For the data format, please see [GitHub repository page](https://github.com/dxda6216/SPARK_fluorescence_anisotropy).

G_factor = 1.000 #@param {type:"number"}
Plotting_data = "No" #@param ["Yes", "No"]

#@markdown 1. Input G-factor
#@markdown 2. **Runtime** -> **Restart and run all** (or press **Ctrl+M** and then press **Ctrl+F9**)
#@markdown 3. Wait until `Choose Files` or `Browse...` button appears below.
#@markdown 4. Click `Choose Files` or `Browse...` button and select a SPARK Excel data file in your computer.
#@markdown 5. Wait a while. Two Excel files, one CSV file, and one ZIP file will be saved in "Downloads" folder in your computer.

### Deleting old files
!rm -f *.xlsx *.csv *.dat *.zip

### Uploading an Excel data file
uploaded = files.upload()

sttime = datetime.now(timezone.utc)
processed_dnt_str = sttime.strftime("%Y-%m-%d %H:%M:%S")
print('\nStarted at '+processed_dnt_str+' (UTC)')

spark_excel_filename = next(iter(uploaded))
print('\n\"'+spark_excel_filename+'\" has been uploaded.')
csv_output_filename = os.path.splitext(spark_excel_filename)[0]+'.csv'
csvdatafile = csv_output_filename
excel_output_filename = os.path.splitext(spark_excel_filename)[0]+'_SPLITTED.xlsx'
excel_output_filename2 = os.path.splitext(spark_excel_filename)[0]+'_Anisotropy_TS.xlsx'
zip_output_filename = os.path.splitext(spark_excel_filename)[0]+'_for_LCA.zip'
print('\nReading the Excel file and generating a CSV file...')
print('\nNew CSV file: \"'+csv_output_filename+'\" will be saved.')
print('New Excel file: \"'+excel_output_filename+'\" will be saved.')
print('New Excel file: \"'+excel_output_filename2+'\" will be saved.')
print('New ZIP file: \"'+zip_output_filename+'\" will be saved.')

### Reading the Excel file and generating a CSV file
read_file = pd.read_excel(spark_excel_filename, header=None, index_col=False)
read_file.to_csv(csv_output_filename, index=None, header=False)

### Finding rows containing each data set
print('\nChecking the structure of the data file...')

def logic_ds(skip_row_number):
	if skip_row_number >= startline and skip_row_number <= endline:
		return False
	return True

f = open(csv_output_filename, "r", encoding="utf-8")
foo_txt = f.read()
f.close()
line_no = 0
for line in foo_txt.splitlines():
	line_no += 1
	if "Label 1 [mP]" in line:
		L1A = line_no
	if "Rawdata (perpendicular)" in line:
		L2A = line_no
	if "Rawdata (parallel)" in line:
		L3A = line_no
	if "Anisotropy" in line:
		L4A = line_no
	if "Total Intensity" in line:
		L5A = line_no
	if "Intensity (parallel)" in line:
		L6A = line_no
	if "Intensity (perpendicular)" in line:
		L7A = line_no
	if "Start Time,,,," in line:
		start_dt_str = line[14:33]
		start_dt = datetime.strptime(start_dt_str, '%Y-%m-%d %H:%M:%S')

abn = L2A - L1A - 3

### Extracting data from Excel/CSV and generating Pandas dataframes
print('\nGenerating Pandas dataframes...')

##### Defining dataframes
Polarization_mP_df = pd.DataFrame()
Rawdata_perpendicular_df = pd.DataFrame()
Rawdata_parallel_df = pd.DataFrame()
Anisotropy_df = pd.DataFrame()
Total_Intensity_df = pd.DataFrame()
Intensity_parallel_df = pd.DataFrame()
Intensity_perpendicular_df = pd.DataFrame()

listofdfs = [
		[ 'Polarization_mP_df', L1A ],
		[ 'Rawdata_perpendicular_df', L2A ],
		[ 'Rawdata_parallel_df', L3A ],
		[ 'Anisotropy_df', L4A ],
		[ 'Total_Intensity_df', L5A ],
		[ 'Intensity_parallel_df', L6A ],
		[ 'Intensity_perpendicular_df', L7A ]
		]

##### Extracting the data to each dataframe
for i in range(7):
	dfname = listofdfs[i][0]
	startline = listofdfs[i][1]
	endline = listofdfs[i][1] + abn
	print(dfname, startline, endline)
	locals()[dfname] = pd.read_csv(csvdatafile, delimiter=',', index_col=False, skiprows = lambda x: logic_ds(x) )
	if i == 0:
		cycleno = locals()[dfname]['Cycle Nr.']
		timesec = locals()[dfname]['Time [s]']
		tmpdc = locals()[dfname]['Temp. [°C]']
		timemin = locals()[dfname]['Time [s]'] / 60
		timehours = locals()[dfname]['Time [s]'] / 3600
		timedays = locals()[dfname]['Time [s]'] / 86400
		spark_dt = start_dt  +  pd.to_timedelta(timesec.astype(int), unit='s')
	locals()[dfname].insert(loc=1, column='DateTime', value=spark_dt)
	locals()[dfname].insert(loc=2, column='Time [hours]', value=timehours)
	locals()[dfname].insert(loc=4, column='Time [m]', value=timemin)
	locals()[dfname].insert(loc=5, column='Time [h]', value=timehours)
	locals()[dfname].insert(loc=6, column='Time [days]', value=timedays)

### Generating a dataframe for time and temp.
Time_and_Temp_df = pd.DataFrame()
Time_and_Temp_df['Cycle Nr.'] = cycleno
Time_and_Temp_df['DateTime'] = spark_dt
Time_and_Temp_df['Time [hours]'] = timehours
Time_and_Temp_df['Time [s]'] = timesec
Time_and_Temp_df['Time [m]'] = timemin
Time_and_Temp_df['Time [h]'] = timehours
Time_and_Temp_df['Time [days]'] = timedays
Time_and_Temp_df['Temp. [°C]'] = tmpdc

### Extracting columns containing data
print('\nExtracting columns containing data...')
Polarization_mP_df2 = Polarization_mP_df.dropna(axis=1,how='all')
list_of_valid_columns = Polarization_mP_df2.columns.tolist()
Rawdata_perpendicular_df2 = Rawdata_perpendicular_df.loc[:, list_of_valid_columns]
Rawdata_parallel_df2 = Rawdata_parallel_df.loc[:, list_of_valid_columns]
Anisotropy_df2 = Anisotropy_df.loc[:, list_of_valid_columns]
Total_Intensity_df2 = Total_Intensity_df.loc[:, list_of_valid_columns]
Intensity_parallel_df2 = Intensity_parallel_df.loc[:, list_of_valid_columns]
Intensity_perpendicular_df2 = Intensity_perpendicular_df.loc[:, list_of_valid_columns]

### Calculating polarization with g-factor
#print('\nCalculating polarization with G-factor...')
#Polarization_mP_float_df = pd.DataFrame(columns=list_of_valid_columns)
#Polarization_mP_float_df['Cycle Nr.'] = cycleno
#Polarization_mP_float_df['DateTime'] = spark_dt
#Polarization_mP_float_df['Time [hours]'] = timehours
#Polarization_mP_float_df['Time [s]'] = timesec
#Polarization_mP_float_df['Time [m]'] = timemin
#Polarization_mP_float_df['Time [h]'] = timehours
#Polarization_mP_float_df['Time [days]'] = timedays
#Polarization_mP_float_df['Temp. [°C]'] = tmpdc
#g_factor = G_factor
#i = 8
#while i < len(list_of_valid_columns):
# 	colname = list_of_valid_columns[i]
# 	i_para = Intensity_parallel_df[colname]
# 	i_perp = Intensity_perpendicular_df[colname]
# 	Polarization_mP_float_df[colname] = 1000 * ( ( i_para - i_perp ) / ( i_para  + g_factor * i_perp ) )
# 	i += 1

### Calculating polarization and anisotropy with g-factor
print('\nCalculating polarization and anisotropy with G-factor...')

Polarization_mP_float_df = pd.DataFrame(columns=list_of_valid_columns)
Polarization_mP_float_df['Cycle Nr.'] = cycleno
Polarization_mP_float_df['DateTime'] = spark_dt
Polarization_mP_float_df['Time [hours]'] = timehours
Polarization_mP_float_df['Time [s]'] = timesec
Polarization_mP_float_df['Time [m]'] = timemin
Polarization_mP_float_df['Time [h]'] = timehours
Polarization_mP_float_df['Time [days]'] = timedays
Polarization_mP_float_df['Temp. [°C]'] = tmpdc

Anisotropy_float_df = pd.DataFrame(columns=list_of_valid_columns)
Anisotropy_float_df['Cycle Nr.'] = cycleno
Anisotropy_float_df['DateTime'] = spark_dt
Anisotropy_float_df['Time [hours]'] = timehours
Anisotropy_float_df['Time [s]'] = timesec
Anisotropy_float_df['Time [m]'] = timemin
Anisotropy_float_df['Time [h]'] = timehours
Anisotropy_float_df['Time [days]'] = timedays
Anisotropy_float_df['Temp. [°C]'] = tmpdc
no_of_time_points = len(cycleno)
g_factor = G_factor
i = 8
while i < len(list_of_valid_columns):
	colname = list_of_valid_columns[i]
	i_para = Rawdata_parallel_df[colname].to_numpy()
	i_perp = Rawdata_perpendicular_df[colname].to_numpy()
	# print(i_para)
	# print(i_perp)
	j = 0
	nvar = None
	Polarization_X = []
	Anisotropy_X = []
	while j < no_of_time_points:
		if i_para[j] == "OVER" or i_perp[j] == "OVER":
			pv = nvar
			av = nvar
		else:
			i_paraj = int(i_para[j])
			i_perpj = int(i_perp[j])
			pv = 1000 * ( ( i_paraj - i_perpj ) / ( i_paraj  + g_factor * i_perpj ) )
			av = 1000 * ( ( i_paraj - i_perpj ) / ( i_paraj  + 2 * g_factor * i_perpj ) )
		# print(colname + ', ' + str(j) + ', ' + str(pv) + ', ' + str(av))
		Polarization_X.append(pv)
		Anisotropy_X.append(av)
		j += 1
	Polarization_mP_float_df[colname] = Polarization_X
	Anisotropy_float_df[colname] = Anisotropy_X
	i += 1

number_of_rows = len(Anisotropy_float_df.index)
last_row_index = number_of_rows - 1
total_time = Anisotropy_float_df.loc[last_row_index]['Time [s]'] - Anisotropy_float_df.loc[0]['Time [s]']
total_time_in_min = total_time * (1/60)
total_time_in_hours = total_time * (1/(60*60))
total_time_in_days = total_time * (1/(24*60*60))
time_interval = total_time_in_hours / last_row_index
time_interval2 = round(time_interval, 12)
print('Number of rows: ' + str(number_of_rows))
print('First time points: ' + str(Anisotropy_float_df.loc[0]['Time [s]']) + ' sec')
print('Last time points: ' + str(Anisotropy_float_df.loc[last_row_index]['Time [s]']) + ' sec')
print('Total time duration: ' + str(total_time) + ' sec = ' + str(total_time_in_min) + ' min = ' + str(total_time_in_hours) + ' hours = ' + str(total_time_in_days) + ' days')
print('Time interval: ' + str(time_interval) + ' h')
print('Time interval: ' + str(time_interval2) + ' h')
excel2_sheet_name = 'Every '+str('{:.12f}'.format(time_interval2))+' h'

Anisotropy_float_df2= Anisotropy_float_df.iloc[:,8:len(list_of_valid_columns)+1]
with pd.ExcelWriter(excel_output_filename2) as writer:
	Anisotropy_float_df2.to_excel(writer, sheet_name=excel2_sheet_name, index=None, header=True)

listofdf2s = [ 'Time_and_Temp_df',
		'Polarization_mP_df2',
		'Rawdata_perpendicular_df2',
		'Rawdata_parallel_df2',
		'Anisotropy_df2',
		'Total_Intensity_df2',
		'Intensity_parallel_df2',
		'Intensity_perpendicular_df2',
		'Polarization_mP_float_df',
		'Anisotropy_float_df' ]

for i in range(10):
	df2name = listofdf2s[i]
	# print('\n'+df2name)
	# print(locals()[df2name])


if Plotting_data == "Yes":
	i = 8
	while i < len(list_of_valid_columns):
		well_to_plot = list_of_valid_columns[i]
		plt.figure(figsize=(10, 4))
		plt.plot(Anisotropy_float_df['Time [hours]'], Anisotropy_float_df[well_to_plot])
		plt.xlabel('Time (hours)')
		plt.ylabel('Fluorescence Anisotropy (mA)')
		plt.title(f'Anisotropy: Well {well_to_plot}')
		plt.grid(True)
		plt.show()
		i += 1

### Generating a new Excel file having multiple sheets
print('\nPreparing a new Excel file to save pre-processed data...')
print('It will take 2-3 min. Wait a while, please.')

##### Generating a worksheet for the note
wells = ''
i = 8
while i < len(list_of_valid_columns):
	colname = list_of_valid_columns[i]
	# if len(colname) == 2:
	# 	wells = wells + colname[:1] + '0' + colname[1:]
	# if len(colname) == 3:
	# 	wells = wells + colname
	wells = wells + colname
	if i < len(list_of_valid_columns) - 1:
		wells = wells + ', '
	i += 1

note_df = pd.DataFrame.from_dict(
		{'A': ['Original Data Filename', 'Processed Date and Time (UTC)', 'G-factor', 'Sample Wells'],
		 'B': ['', '', '', ''],
		 'C': ['', '', '', ''],
		 'D': ['', '', '', ''],
		 'E': [spark_excel_filename, processed_dnt_str, g_factor, wells]
		 })

##### Writting the data to the new Excel file
with pd.ExcelWriter(excel_output_filename) as writer:
	read_file.to_excel(writer, sheet_name='Spark Datasheet', index=None, header=False)
	note_df.to_excel(writer, sheet_name='Note', index=None, header=False)
	Time_and_Temp_df.to_excel(writer, sheet_name='Time and Temp')
	Polarization_mP_df2.to_excel(writer, sheet_name='Polarization (mP)')
	Rawdata_perpendicular_df2.to_excel(writer, sheet_name='Rawdata Perpendicular')
	Rawdata_parallel_df2.to_excel(writer, sheet_name='Rawdata Parallel')
	Anisotropy_df2.to_excel(writer, sheet_name='Anisotropy')
	Total_Intensity_df2.to_excel(writer, sheet_name='Total Intensity')
	Intensity_parallel_df2.to_excel(writer, sheet_name='Intensity Parallel')
	Intensity_perpendicular_df2.to_excel(writer, sheet_name='Intensity Perpendicular')
	Polarization_mP_float_df.to_excel(writer, sheet_name='Polarization corr by G factor')
	Anisotropy_float_df.to_excel(writer, sheet_name='Anisotropy corr by G factor')

### Generating data files for LumiCycle Analysis
print('\nPreparing an anisotropy data file (readable with LumiCycle Analysis) for each well...')
i = 8
while i < len(list_of_valid_columns):
	colname = list_of_valid_columns[i]
	if len(colname) == 2:
		outputdatfilename = colname[:1] + '0' + colname[1:] + '.dat'
	if len(colname) == 3:
		outputdatfilename = colname + '.dat'
	Anisotropy_float_df.to_csv(outputdatfilename, header=False, index=False, sep ='\t', columns=['Time [days]',colname])
	print(outputdatfilename)
	i += 1

##### Packing the .dat files into a zip file
print('\nPacking the .dat files into a zip file...')
!zip -r {zip_output_filename} ./*.dat

### Downloading the CSV, Excel, and ZIP files into the local host
files.download(csv_output_filename)
files.download(excel_output_filename)
files.download(excel_output_filename2)
files.download(zip_output_filename)

print('\nDownloading the CSV, Excel, and Zip files into \"Downloads\" folder in your computer...')

endtime = datetime.now(timezone.utc)
td_m = endtime - sttime
processed_dnt_str = endtime.strftime("%Y-%m-%d %H:%M:%S")
print('\nElapse time: '+str(td_m.seconds)+' seconds')
print('\nCompleted at '+processed_dnt_str+' (UTC)\n')

### End of script